In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[16]: # Gardez vos numéros de cellule si vous travaillez dans un notebook


#!/usr/bin/env python
# coding: utf-8

import configparser
import logging
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import load
import winsound # Vous avez ajouté winsound

# Importer PIDController si défini dans un autre module, sinon le copier ici.
# from pid_controller_module import PIDController

# --- Configuration du Logging ---
logger = logging.getLogger(__name__)

try:
    # Essayer d'obtenir le nom du fichier depuis __file__ (pour exécution en script)
    NOM_SCRIPT_SANS_EXTENSION = Path(__file__).stem
except NameError:
    # Fallback si __file__ n'est pas défini (ex: notebook Jupyter)
    NOM_SCRIPT_SANS_EXTENSION = "PID_Tuner_Gemini" # Définit manuellement
    # Le logger n'est pas encore configuré ici pour .info, mais print est sûr
    print(f"INFO (pré-config logger): La variable __file__ n'est pas définie. Utilisation de '{NOM_SCRIPT_SANS_EXTENSION}' comme nom de base par défaut.")

# --- Classe PIDController (identique à votre version) ---
# ... (collez ici votre classe PIDController complète et correcte)
class PIDController:
    def __init__(self, Kp, Ti, Td, Tsamp, mv_min, mv_max, direct_action=True, initial_mv=0.0):
        self.Kp_param = Kp
        self.Ti_param = float('inf') if Ti <= 0 or Ti == float('inf') else Ti
        self.Td_param = Td
        self.Tsamp = Tsamp  # en secondes
        self.mv_min = mv_min
        self.mv_max = mv_max
        self.direct_action = direct_action
        self.proportional_term = 0.0
        self.integral_term = 0.0
        self.derivative_term = 0.0
        self.previous_pv = None
        self.previous_error = None
        self.mv = initial_mv
        self.last_active_mv = initial_mv
        self._update_internal_gains()

    def _update_internal_gains(self):
        self.kp_calc = self.Kp_param
        if self.Ti_param == float('inf') or self.Tsamp <= 0:
            self.ki_calc = 0.0
        else:
            self.ki_calc = (self.Kp_param * self.Tsamp) / self.Ti_param
        if self.Tsamp <= 0:
            self.kd_calc = 0.0
        else:
            self.kd_calc = (self.Kp_param * self.Td_param) / self.Tsamp

    def set_parameters(self, Kp, Ti, Td):
        parameter_changed = False
        if self.Kp_param != Kp: self.Kp_param = Kp; parameter_changed = True
        new_ti_param = float('inf') if Ti <= 0 or Ti == float('inf') else Ti
        if self.Ti_param != new_ti_param: self.Ti_param = new_ti_param; parameter_changed = True
        if self.Td_param != Td: self.Td_param = Td; parameter_changed = True
        if parameter_changed:
            self._update_internal_gains()
            logger.debug(f"Paramètres PID mis à jour : Kp={self.Kp_param}, Ti={self.Ti_param}, Td={self.Td_param}")
            logger.debug(f"Gains internes : kp_calc={self.kp_calc}, ki_calc={self.ki_calc}, kd_calc={self.kd_calc}")

    def set_initial_state(self, pv_initial, sp_initial, mv_initial):
        self.previous_pv = pv_initial
        self.mv = self._limit_mv(mv_initial)
        self.last_active_mv = self.mv
        error = sp_initial - pv_initial
        if not self.direct_action: error = -error
        self.previous_error = error
        self.proportional_term = self.kp_calc * error
        self.derivative_term = 0.0
        self.integral_term = self.mv - (self.proportional_term + self.derivative_term)
        logger.info(f"État initial du PID : PV={pv_initial:.2f}, SP={sp_initial:.2f}, MV={self.mv:.2f}")
        logger.debug(f"Termes initiaux : P={self.proportional_term:.2f}, I={self.integral_term:.2f}, D={self.derivative_term:.2f}")

    def _limit_mv(self, mv_candidate):
        return max(self.mv_min, min(mv_candidate, self.mv_max))

    def update(self, sp, pv):
        if self.previous_pv is None:
            self.previous_pv = pv
            if self.previous_error is None:
                 error_for_first_D = sp - pv
                 if not self.direct_action: error_for_first_D = -error_for_first_D
                 self.previous_error = error_for_first_D
        error = sp - pv
        if not self.direct_action: error = -error
        self.proportional_term = self.kp_calc * error
        self.derivative_term = 0.0
        if self.kd_calc > 0 and self.previous_pv is not None:
            delta_pv = pv - self.previous_pv
            self.derivative_term = -self.kd_calc * delta_pv
        mv_sans_increment_integral = self.proportional_term + self.integral_term + self.derivative_term
        integral_increment = 0.0
        if self.ki_calc > 0:
            integral_increment = self.ki_calc * error
        mv_candidate = mv_sans_increment_integral + integral_increment
        mv_limitee = self._limit_mv(mv_candidate)
        if self.ki_calc > 0 :
            if mv_candidate != mv_limitee:
                self.integral_term = mv_limitee - (self.proportional_term + self.derivative_term)
            else:
                self.integral_term += integral_increment
        self.mv = mv_limitee
        self.previous_pv = pv
        self.previous_error = error
        return self.mv

# --- Fonctions de Configuration et Utilitaires (identique à votre version) ---
# ... (collez ici votre fonction load_config_and_setup_logging complète et correcte)
# ... (collez ici votre fonction load_process_model_and_scalers complète et correcte)
# ... (collez ici votre fonction get_model_feature_names_from_config complète et correcte)
# ... (collez ici votre fonction prepare_model_input_frame complète et correcte)
def load_config_and_setup_logging(config_file_path_str):
    config_path = Path(config_file_path_str)
    if not config_path.is_file():
        print(f"ERREUR CRITIQUE: Fichier de configuration '{config_path}' non trouvé.")
        sys.exit(1)
    config = configparser.ConfigParser(inline_comment_prefixes=(';', '#'))
    config.optionxform = str
    config.read(config_path, encoding='utf-8')
    log_level_str = config.get('General', 'log_level', fallback='INFO').upper()
    numeric_log_level = getattr(logging, log_level_str, logging.INFO)
    log_file_base = config.get('General', 'log_file_base_name', fallback=NOM_SCRIPT_SANS_EXTENSION)
    log_file_name = f"{log_file_base}.txt"
    log_file_path = Path(log_file_name).resolve()
    log_file_path.parent.mkdir(parents=True, exist_ok=True)
    current_logger = logging.getLogger(__name__)
    if current_logger.hasHandlers():
        for handler in current_logger.handlers[:]:
            try: handler.close()
            except Exception: pass
            current_logger.removeHandler(handler)
    current_logger.setLevel(numeric_log_level)
    current_logger.propagate = False
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s')
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    try:
        file_handler = logging.FileHandler(log_file_path, mode='w', encoding='utf-8')
        file_handler.setFormatter(file_formatter)
        file_handler.setLevel(numeric_log_level)
        current_logger.addHandler(file_handler)
    except Exception as e:
        print(f"ERREUR CRITIQUE lors de la création du file_handler pour le log : {e}", file=sys.stderr)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(console_formatter)
    console_handler.setLevel(numeric_log_level)
    current_logger.addHandler(console_handler)
    logger.info(f"Logging configuré. Niveau: {log_level_str}. Fichier log: {log_file_path}")
    logger.info(f"Configuration chargée depuis '{config_path}'.")
    return config

def load_process_model_and_scalers(config_modele):
    model_path_str = config_modele.get('model_path')
    scalers_path_str = config_modele.get('scalers_path')
    if not model_path_str or not scalers_path_str:
        logger.error("Chemin du modèle ou des scalers non spécifié.")
        raise ValueError("Chemin modèle/scalers manquant.")
    model_path = Path(model_path_str); scalers_path = Path(scalers_path_str)
    if not model_path.is_file(): raise FileNotFoundError(f"Modèle non trouvé : {model_path}")
    if not scalers_path.is_file(): raise FileNotFoundError(f"Scalers non trouvés : {scalers_path}")
    try:
        process_model = load(model_path)
        scalers_dict = load(scalers_path)
        logger.info(f"Modèle chargé: {model_path}, Scalers chargés: {scalers_path}")
        return process_model, scalers_dict['scaler_X'], scalers_dict['scaler_y']
    except Exception as e: logger.error(f"Erreur chargement modèle/scalers: {e}", exc_info=True); raise

def get_model_feature_names_from_config(config_modele_features):
    feature_names = []
    lag_config_order = [('pv_lags', 'PV'), ('mv_lags', 'MV'), ('sp_lags', 'SP'),
                        ('kp_hist_lags', 'Kp_hist'), ('ti_hist_lags', 'Ti_hist'), ('td_hist_lags', 'Td_hist')]
# Dans la fonction get_model_feature_names_from_config:
# ... (lag_config_order est défini avant) ...

    # Ajouter les perturbations dynamiquement
    i = 1
    max_disturbances_to_check = 10 # Sécurité pour éviter boucle infinie
    while i <= max_disturbances_to_check: 
        dist_lag_key = f'disturbance_{i}_lags'

        if config_modele_features.has_option(dist_lag_key):
            dist_lag_value_str = config_modele_features.get(dist_lag_key, '').strip()
            # Vérifier si la clé de configuration pour les lags a une valeur et que cette valeur est > 0
            if dist_lag_value_str and config_modele_features.getint(dist_lag_key, 0) > 0:
                lag_config_order.append((dist_lag_key, f'Dist{i}'))
                logger.debug(f"Utilisation de {dist_lag_key} (valeur: {dist_lag_value_str}) pour Dist{i} dans le fallback.")
            else:
                logger.debug(f"La clé {dist_lag_key} existe mais est vide ou lag=0 (fallback). Passage à la suivante.")
        else:
            logger.debug(f"La clé {dist_lag_key} n'existe pas (fallback). Arrêt de la recherche de lags de perturbation.")
            break 
        i += 1

    if i > max_disturbances_to_check: # Vérification si on a atteint la limite
        logger.warning(f"Limite de {max_disturbances_to_check} perturbations vérifiées atteinte dans le fallback.")

    feature_names = [] # Doit être défini ici
    for config_lag_key_loop, base_col_name_loop in lag_config_order:
        num_lags = config_modele_features.getint(config_lag_key_loop, 0)
        if num_lags > 0:
            for lag_idx in range(1, num_lags + 1):
                feature_names.append(f'{base_col_name_loop}_lag_{lag_idx}')

    logger.info(f"Noms des features du modèle (générés depuis config - fallback, ordre important) : {feature_names}")
    if not feature_names:
        logger.warning("Aucun nom de feature généré depuis la config (fallback). Vérifiez [ModeleProcede].")
    return feature_names

def prepare_model_input_frame(current_history_df, feature_names_ordered_list, config_modele_features):
    model_input_dict = {}
    for full_feature_name in feature_names_ordered_list:
        parts = full_feature_name.split('_lag_')
        if len(parts) == 2:
            base_col_name = parts[0]; lag_idx = int(parts[1])
            if base_col_name not in current_history_df.columns: model_input_dict[full_feature_name] = np.nan; logger.warning(f"Col base '{base_col_name}' pour '{full_feature_name}' absente history_df -> NaN."); continue
            if len(current_history_df) >= lag_idx: model_input_dict[full_feature_name] = current_history_df[base_col_name].iloc[-lag_idx]
            else: model_input_dict[full_feature_name] = np.nan; logger.error(f"Hist insuffisant pour '{full_feature_name}'. Requis:{lag_idx}, Dispo:{len(current_history_df)} -> NaN.")
        elif full_feature_name in current_history_df.columns: model_input_dict[full_feature_name] = current_history_df[full_feature_name].iloc[-1]
        else: model_input_dict[full_feature_name] = np.nan; logger.warning(f"Feature '{full_feature_name}' non gérée ou absente history_df -> NaN.")
    single_row_data = {fn: [model_input_dict.get(fn, np.nan)] for fn in feature_names_ordered_list}
    final_input_df = pd.DataFrame(single_row_data, columns=feature_names_ordered_list)
    if final_input_df.isnull().values.any(): logger.warning(f"NaNs dans entrée modèle: {final_input_df.columns[final_input_df.isnull().any()].tolist()}.")
    return final_input_df

# --- Noyau de Simulation ---
def run_closed_loop_simulation(config, nom_jeu_reglage, Kp, Ti, Td,
                               modele_procede, scaler_X, scaler_y,
                               model_feature_names):
    logger.info(f"--- Démarrage simulation pour jeu : {nom_jeu_reglage} (Kp={Kp}, Ti={Ti}, Td={Td}) ---")

    cfg_pid_base = config['ParametresPIDBase']
    cfg_sim_scenario = config['ScenarioSimulation']
    cfg_modele_features = config['ModeleProcede']

    tsamp_s = cfg_pid_base.getfloat('tsamp_pid_sim_ms') / 1000.0
    mv_min_sim = cfg_pid_base.getfloat('mv_min')
    mv_max_sim = cfg_pid_base.getfloat('mv_max')
    direct_action_sim = cfg_pid_base.getboolean('direct_action')

    sim_duration_s = cfg_sim_scenario.getfloat('simulation_duration_seconds')
    num_steps = int(sim_duration_s / tsamp_s)

    pv_initiale = cfg_sim_scenario.getfloat('initial_pv')
    mv_initiale = cfg_sim_scenario.getfloat('initial_mv')
    
    pid = PIDController(Kp, Ti, Td, tsamp_s, mv_min_sim, mv_max_sim, direct_action_sim, initial_mv=mv_initiale)

    time_points = np.arange(0, sim_duration_s, tsamp_s)[:num_steps]
    
    # Préparation du profil de SP
    valeurs_sp = np.full_like(time_points, cfg_sim_scenario.getfloat('sp_constant_value', pv_initiale)) # Défaut à pv_initiale si non défini
    type_sp = cfg_sim_scenario.get('setpoint_type', 'constant')
    if type_sp == 'step':
        steps_str = cfg_sim_scenario.get('sp_steps', '')
        val_sp_actuelle = cfg_sim_scenario.getfloat('initial_pv') # SP initiale = PV initiale par défaut avant le 1er step
        if steps_str:
            parsed_steps = []
            for step_pair_str in steps_str.split(';'):
                time_val, sp_val = map(str.strip, step_pair_str.split(','))
                parsed_steps.append((float(time_val), float(sp_val)))
            parsed_steps.sort()
            if parsed_steps and parsed_steps[0][0] == 0:
                val_sp_actuelle = parsed_steps[0][1]
            
            idx_step = 0
            for i, t_sim_current in enumerate(time_points):
                while idx_step < len(parsed_steps) and t_sim_current >= parsed_steps[idx_step][0]:
                    val_sp_actuelle = parsed_steps[idx_step][1]
                    idx_step += 1
                valeurs_sp[i] = val_sp_actuelle
    sp_initiale_pour_pid = valeurs_sp[0]

    # Préparation du profil pour Dist1
    valeurs_dist1 = np.full_like(time_points, cfg_sim_scenario.getfloat('disturbance1_initial_value', 0.0))
    type_dist1 = cfg_sim_scenario.get('disturbance1_type', 'constant')
    if type_dist1 == 'constant':
        valeurs_dist1[:] = cfg_sim_scenario.getfloat('disturbance1_constant_value', 0.0)
    elif type_dist1 == 'step':
        steps_dist1_str = cfg_sim_scenario.get('disturbance1_steps', '')
        val_dist1_actuelle = cfg_sim_scenario.getfloat('disturbance1_initial_value', 0.0)
        if steps_dist1_str:
            parsed_steps_dist1 = []
            for step_pair_str in steps_dist1_str.split(';'):
                time_val, dist_val = map(str.strip, step_pair_str.split(','))
                parsed_steps_dist1.append((float(time_val), float(dist_val)))
            parsed_steps_dist1.sort()
            if parsed_steps_dist1 and parsed_steps_dist1[0][0] == 0:
                val_dist1_actuelle = parsed_steps_dist1[0][1]

            idx_step_dist1 = 0
            for i, t_sim_current in enumerate(time_points):
                while idx_step_dist1 < len(parsed_steps_dist1) and t_sim_current >= parsed_steps_dist1[idx_step_dist1][0]:
                    val_dist1_actuelle = parsed_steps_dist1[idx_step_dist1][1]
                    idx_step_dist1 += 1
                valeurs_dist1[i] = val_dist1_actuelle
    dist1_initiale_pour_historique = valeurs_dist1[0]

    # Préparation du profil pour Dist2 (NOUVEAU)
    valeurs_dist2 = np.full_like(time_points, cfg_sim_scenario.getfloat('disturbance2_initial_value', 0.0))
    type_dist2 = cfg_sim_scenario.get('disturbance2_type', 'constant')
    if type_dist2 == 'constant':
        valeurs_dist2[:] = cfg_sim_scenario.getfloat('disturbance2_constant_value', 0.0)
    elif type_dist2 == 'step':
        steps_dist2_str = cfg_sim_scenario.get('disturbance2_steps', '')
        val_dist2_actuelle = cfg_sim_scenario.getfloat('disturbance2_initial_value', 0.0)
        if steps_dist2_str:
            parsed_steps_dist2 = []
            for step_pair_str in steps_dist2_str.split(';'):
                time_val, dist_val = map(str.strip, step_pair_str.split(','))
                parsed_steps_dist2.append((float(time_val), float(dist_val)))
            parsed_steps_dist2.sort()
            if parsed_steps_dist2 and parsed_steps_dist2[0][0] == 0:
                val_dist2_actuelle = parsed_steps_dist2[0][1]
            
            idx_step_dist2 = 0
            for i, t_sim_current in enumerate(time_points):
                while idx_step_dist2 < len(parsed_steps_dist2) and t_sim_current >= parsed_steps_dist2[idx_step_dist2][0]:
                    val_dist2_actuelle = parsed_steps_dist2[idx_step_dist2][1]
                    idx_step_dist2 += 1
                valeurs_dist2[i] = val_dist2_actuelle
    dist2_initiale_pour_historique = valeurs_dist2[0]
    # FIN NOUVEAU pour Dist2

    pid.set_initial_state(pv_initiale, sp_initiale_pour_pid, mv_initiale)
    pv_actuelle = pv_initiale
    
    max_lag = 0
    if model_feature_names:
        for fname in model_feature_names:
            if "_lag_" in fname:
                try: max_lag = max(max_lag, int(fname.split('_lag_')[-1]))
                except ValueError: pass
    else: 
        for key_cfg_lag in ['pv_lags', 'mv_lags', 'sp_lags', 'kp_hist_lags', 'ti_hist_lags', 'td_hist_lags']: # et disturbances
            max_lag = max(max_lag, cfg_modele_features.getint(key_cfg_lag, 0))
        for i in range(1, 4): # Chercher Dist1, Dist2, Dist3 lags
            key_cfg_dist_lag = f'disturbance_{i}_lags'
            if cfg_modele_features.has_option(key_cfg_dist_lag):
                 max_lag = max(max_lag, cfg_modele_features.getint(key_cfg_dist_lag, 0))

    longueur_historique = max_lag if max_lag > 0 else 1
    logger.debug(f"Max lag requis: {max_lag}. Longueur buffer historique: {longueur_historique}")

    cols_base_historique = set()
    for fname in model_feature_names:
        if "_lag_" in fname: cols_base_historique.add(fname.split('_lag_')[0])
        else: cols_base_historique.add(fname)
    cols_base_historique.update(['PV', 'MV', 'SP']) # S'assurer qu'elles y sont pour la logique
    # Ajouter Dist1 et Dist2 si le modèle les utilise ou si on veut les historiser pour une raison quelconque
    if cfg_modele_features.getint('disturbance_1_lags', 0) > 0 or any("Dist1" in s for s in model_feature_names):
        cols_base_historique.add('Dist1')
    if cfg_modele_features.getint('disturbance_2_lags', 0) > 0 or any("Dist2" in s for s in model_feature_names): # NOUVEAU
        cols_base_historique.add('Dist2')                                                                      # NOUVEAU


    colonnes_df_historique = sorted(list(cols_base_historique))
    logger.debug(f"Colonnes de base pour df_historique : {colonnes_df_historique}")
    
    donnees_historique_init = {}
    if 'PV' in colonnes_df_historique: donnees_historique_init['PV'] = [pv_initiale] * longueur_historique
    if 'MV' in colonnes_df_historique: donnees_historique_init['MV'] = [mv_initiale] * longueur_historique
    if 'SP' in colonnes_df_historique: donnees_historique_init['SP'] = [sp_initiale_pour_pid] * longueur_historique
    if 'Dist1' in colonnes_df_historique:
        donnees_historique_init['Dist1'] = [dist1_initiale_pour_historique] * longueur_historique
        logger.info(f"Col 'Dist1' initialisée à {dist1_initiale_pour_historique} pour lags initiaux.")
    if 'Dist2' in colonnes_df_historique: # NOUVEAU
        donnees_historique_init['Dist2'] = [dist2_initiale_pour_historique] * longueur_historique
        logger.info(f"Col 'Dist2' initialisée à {dist2_initiale_pour_historique} pour lags initiaux.")

    for col_hist in colonnes_df_historique:
        if col_hist not in donnees_historique_init:
             logger.warning(f"Col '{col_hist}' non explicitement initialisée. Défaut à 0.0 pour lags.")
             donnees_historique_init[col_hist] = [0.0] * longueur_historique
    
    df_historique = pd.DataFrame(donnees_historique_init, columns=colonnes_df_historique)
    resultats_sim = {'Time': [], 'SP': [], 'PV': [], 'MV': [], 'Error': [], 'Dist1': [], 'Dist2': []} # Ajout Dist1, Dist2 aux résultats
    logger.debug(f"df_historique initial pour entrées modèle :\n{df_historique.head().to_string()}")

    for i_step in range(num_steps):
        t_actuel = time_points[i_step]
        sp_actuelle = valeurs_sp[i_step]
        dist1_actuelle = valeurs_dist1[i_step] # Récupérer la valeur de Dist1 pour ce pas
        dist2_actuelle = valeurs_dist2[i_step] # NOUVEAU: Récupérer la valeur de Dist2

        mv_actuelle = pid.update(sp_actuelle, pv_actuelle)
        input_X_df_modele = prepare_model_input_frame(df_historique, model_feature_names, cfg_modele_features)
        
        if input_X_df_modele.isnull().values.any():
            cols_nan_input = input_X_df_modele.columns[input_X_df_modele.isnull().any()].tolist()
            logger.error(f"Étape {i_step}, T {t_actuel:.2f}s: NaNs entrée modèle pour {cols_nan_input}. Arrêt simu.")
            remaining_steps = num_steps - i_step
            for key_res in resultats_sim.keys(): resultats_sim[key_res].extend([np.nan] * remaining_steps)
            break
            
        input_X_modele_scaled = scaler_X.transform(input_X_df_modele)
        pv_predite_scaled = modele_procede.predict(input_X_modele_scaled)
        pv_suivante = scaler_y.inverse_transform(pv_predite_scaled.reshape(-1, 1)).ravel()[0]
        logger.debug(f"Étape {i_step}: Scaler_y info: data_min={scaler_y.data_min_[0]:.4f}, data_range={scaler_y.data_range_[0]:.4f}")
        logger.debug(f"Étape {i_step}: predicted_pv_scaled: {pv_predite_scaled[0]:.6f}, pv_suivante (déscalé): {pv_suivante:.6f}")
        logger.debug(f"Étape {i_step}: input_X_df_modele (avant scale):\n{input_X_df_modele.head().to_string()}")
        logger.debug(f"Étape {i_step}: input_X_modele_scaled (1ere ligne):\n{input_X_modele_scaled[0]}")
        logger.debug(f"Étape {i_step}: predicted_pv_scaled: {pv_predite_scaled[0]}, pv_suivante (déscalé): {pv_suivante}")
        
        resultats_sim['Time'].append(t_actuel)
        resultats_sim['SP'].append(sp_actuelle)
        resultats_sim['PV'].append(pv_actuelle)
        resultats_sim['MV'].append(mv_actuelle)
        resultats_sim['Error'].append(sp_actuelle - pv_actuelle)
        resultats_sim['Dist1'].append(dist1_actuelle) # Enregistrer Dist1
        resultats_sim['Dist2'].append(dist2_actuelle) # NOUVEAU: Enregistrer Dist2

        pv_actuelle = pv_suivante
        
        donnees_nouvelle_ligne_historique = {}
        if 'PV' in colonnes_df_historique: donnees_nouvelle_ligne_historique['PV'] = pv_actuelle
        if 'MV' in colonnes_df_historique: donnees_nouvelle_ligne_historique['MV'] = mv_actuelle
        if 'SP' in colonnes_df_historique: donnees_nouvelle_ligne_historique['SP'] = sp_actuelle
        if 'Dist1' in colonnes_df_historique: donnees_nouvelle_ligne_historique['Dist1'] = dist1_actuelle # Mettre à jour Dist1 dans l'historique
        if 'Dist2' in colonnes_df_historique: donnees_nouvelle_ligne_historique['Dist2'] = dist2_actuelle # NOUVEAU

        for col_hist_update in colonnes_df_historique:
             if col_hist_update not in donnees_nouvelle_ligne_historique:
                  donnees_nouvelle_ligne_historique[col_hist_update] = df_historique[col_hist_update].iloc[-1]
        
        nouvelle_ligne_historique = pd.DataFrame([donnees_nouvelle_ligne_historique], columns=colonnes_df_historique)
        df_historique = pd.concat([df_historique.iloc[1:], nouvelle_ligne_historique], ignore_index=True)
        
        if i_step < 5 or i_step == num_steps -1 or i_step % (max(1, num_steps // 10)) == 0:
             logger.debug(f"t={t_actuel:.2f}s, SP={sp_actuelle:.2f}, PV(in)={resultats_sim['PV'][-1]:.2f}, "
                          f"MV={mv_actuelle:.2f}, PV(pred_out)={pv_actuelle:.2f}, "
                          f"Dist1={dist1_actuelle:.1f}, Dist2={dist2_actuelle:.1f}") # Ajout Dist1, Dist2 au log

    logger.info(f"--- Simulation terminée pour le jeu : {nom_jeu_reglage} ---")
    return pd.DataFrame(resultats_sim)

# --- Calcul des Métriques de Performance (pas de changement ici) ---
# ... (collez ici votre fonction calculate_performance_metrics complète et correcte)
def calculate_performance_metrics(df_resultats, tsamp_s):
    if df_resultats.empty or len(df_resultats) < 2 or df_resultats['PV'].isnull().all():
        logger.warning("DataFrame résultats vide/PV NaN. Métriques non calculables.")
        return {'IAE': np.nan, 'ISE': np.nan, 'ITAE': np.nan, 'Overshoot': np.nan, 'TempsStabilisation': np.nan, 'TempsMontee': np.nan}
    df_resultats_valides = df_resultats.dropna(subset=['Error', 'PV', 'SP', 'Time'])
    if len(df_resultats_valides) < 2 :
        logger.warning("Pas assez données valides post dropna pour métriques.")
        return {'IAE': np.nan, 'ISE': np.nan, 'ITAE': np.nan, 'Overshoot': np.nan, 'TempsStabilisation': np.nan, 'TempsMontee': np.nan}
    erreur = df_resultats_valides['Error']; pv_vals = df_resultats_valides['PV']; sp_vals = df_resultats_valides['SP']; temps_vals = df_resultats_valides['Time']
    iae = np.sum(np.abs(erreur)) * tsamp_s; ise = np.sum(erreur**2) * tsamp_s; itae = np.sum(temps_vals * np.abs(erreur)) * tsamp_s
    metriques = {'IAE': iae, 'ISE': ise, 'ITAE': itae}
    sp_diff = sp_vals.diff().abs()
    if sp_diff.max() > 1e-6:
        last_major_step_idx = sp_diff[sp_diff > 1e-6].index[-1] if not sp_diff[sp_diff > 1e-6].empty else 0
        iloc_idx = df_resultats_valides.index.get_loc(last_major_step_idx) if isinstance(last_major_step_idx, type(df_resultats_valides.index[0])) else last_major_step_idx
        sp_avant_echelon = pv_vals.iloc[iloc_idx-1] if iloc_idx > 0 else pv_vals.iloc[0]
        pv_avant_echelon = pv_vals.iloc[iloc_idx-1] if iloc_idx > 0 else pv_vals.iloc[0]
        sp_apres_echelon = sp_vals.iloc[iloc_idx]
        pv_reponse = pv_vals.iloc[iloc_idx:]; temps_reponse = temps_vals.iloc[iloc_idx:] - temps_vals.iloc[iloc_idx]
        delta_sp = sp_apres_echelon - pv_avant_echelon
        if abs(delta_sp) > 1e-6 :
            if delta_sp > 0: 
                overshoot_val = (pv_reponse.max() - sp_apres_echelon) / abs(delta_sp) * 100; metriques['Overshoot'] = max(0, overshoot_val)
                try:
                    cible_10_pc = pv_avant_echelon + 0.1 * delta_sp; cible_90_pc = pv_avant_echelon + 0.9 * delta_sp
                    temps_10_pc = temps_reponse[pv_reponse >= cible_10_pc].iloc[0]; temps_90_pc = temps_reponse[pv_reponse >= cible_90_pc].iloc[0]
                    metriques['TempsMontee'] = temps_90_pc - temps_10_pc
                except IndexError: metriques['TempsMontee'] = np.nan
            else: 
                undershoot_val = (sp_apres_echelon - pv_reponse.min()) / abs(delta_sp) * 100; metriques['Overshoot'] = max(0, undershoot_val)
                try:
                    cible_10_pc_desc = pv_avant_echelon + 0.1 * delta_sp; cible_90_pc_desc = pv_avant_echelon + 0.9 * delta_sp
                    tps_90_val_init = temps_reponse[pv_reponse <= cible_90_pc_desc].iloc[0]; tps_10_val_init = temps_reponse[pv_reponse <= cible_10_pc_desc].iloc[0]
                    metriques['TempsMontee'] = tps_10_val_init - tps_90_val_init
                except IndexError: metriques['TempsMontee'] = np.nan
            bande_sup_stab = sp_apres_echelon * 1.02; bande_inf_stab = sp_apres_echelon * 0.98
            est_stabilise_mask = (pv_reponse >= bande_inf_stab) & (pv_reponse <= bande_sup_stab)
            temps_stabilisation_val = np.nan
            for k_stab in range(len(pv_reponse) -1, -1, -1):
                if not est_stabilise_mask.iloc[k_stab]:
                    temps_stabilisation_val = temps_reponse.iloc[k_stab+1] if k_stab + 1 < len(temps_reponse) else temps_reponse.iloc[-1]; break
            if np.isnan(temps_stabilisation_val) and not est_stabilise_mask.empty and est_stabilise_mask.iloc[0]: temps_stabilisation_val = temps_reponse.iloc[0]
            metriques['TempsStabilisation'] = temps_stabilisation_val
        else: metriques['Overshoot'] = 0.0; metriques['TempsMontee'] = 0.0; metriques['TempsStabilisation'] = 0.0
    else: metriques['Overshoot'] = np.nan; metriques['TempsMontee'] = np.nan; metriques['TempsStabilisation'] = np.nan
    for k, v in metriques.items(): logger.info(f"Métrique {k}: {v:.3f}" if isinstance(v, float) else f"Métrique {k}: {v}")
    return metriques

# --- Fonctions de Tracé des Résultats ---
def plot_simulation_results(tous_dfs_resultats, noms_jeux_reglage, config_sortie):
    num_jeux = len(tous_dfs_resultats)
    if num_jeux == 0: logger.warning("Aucun résultat à tracer."); return
    # NOUVEAU: Vérifier si Dist1 et Dist2 sont dans les résultats pour les tracer
    has_dist1 = 'Dist1' in tous_dfs_resultats[0].columns
    has_dist2 = 'Dist2' in tous_dfs_resultats[0].columns
    
    num_subplots = 2
    if has_dist1: num_subplots +=1
    if has_dist2 and has_dist1 and 'Dist2' != 'Dist1' : num_subplots +=1 # Eviter de créer un subplot si Dist1=Dist2
    elif has_dist2 and not has_dist1: num_subplots +=1


    fig, axs = plt.subplots(num_subplots, 1, figsize=(18, 5 * num_subplots), sharex=True)
    # S'assurer que axs est toujours une liste, même avec 1 subplot (ne devrait pas arriver ici)
    if num_subplots == 1 : axs = [axs] 

    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']

    current_ax_idx = 0

    # Tracé PV/SP
    for i, df_resultats in enumerate(tous_dfs_resultats):
        if df_resultats.empty or df_resultats['PV'].isnull().all(): continue
        nom_jeu = noms_jeux_reglage[i]
        color = colors[i % len(colors)]
        axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['PV'], label=f'PV ({nom_jeu})', color=color, linewidth=1.5)
        if i == 0: 
            axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['SP'], 'k--', label='Consigne (SP)', alpha=0.8, linewidth=2)
    axs[current_ax_idx].set_ylabel('Variable de Procédé (PV)')
    axs[current_ax_idx].legend(loc='best', fontsize='small')
    axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
    axs[current_ax_idx].set_title('Comparaison des Réglages PID') # Titre sur le premier subplot
    current_ax_idx += 1

    # Tracé MV
    for i, df_resultats in enumerate(tous_dfs_resultats):
        if df_resultats.empty or df_resultats['PV'].isnull().all(): continue # Utiliser PV comme indicateur de validité du run
        nom_jeu = noms_jeux_reglage[i]
        color = colors[i % len(colors)]
        axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['MV'], label=f'MV ({nom_jeu})', color=color, linewidth=1.5)
    axs[current_ax_idx].set_ylabel('Variable Manipulée (MV)')
    axs[current_ax_idx].legend(loc='best', fontsize='small')
    axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
    current_ax_idx += 1

    # Tracé Dist1 (si présente)
    if has_dist1:
        # On ne trace Dist1 qu'une fois (supposée la même pour tous les jeux)
        # ou si elle diffère, il faudrait une boucle comme pour PV/MV
        axs[current_ax_idx].plot(tous_dfs_resultats[0]['Time'], tous_dfs_resultats[0]['Dist1'], label='Disturbance 1 (Dist1)', color='purple', linestyle='-.')
        axs[current_ax_idx].set_ylabel('Disturbance 1')
        axs[current_ax_idx].legend(loc='best', fontsize='small')
        axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
        current_ax_idx +=1

    # Tracé Dist2 (si présente et différente de Dist1 si Dist1 aussi tracée)
    if has_dist2:
        if not (has_dist1 and 'Dist2' == 'Dist1'): # Simple check pour éviter redondance si Dist1 et Dist2 sont la même colonne
            axs[current_ax_idx].plot(tous_dfs_resultats[0]['Time'], tous_dfs_resultats[0]['Dist2'], label='Disturbance 2 (Dist2)', color='brown', linestyle=':')
            axs[current_ax_idx].set_ylabel('Disturbance 2')
            axs[current_ax_idx].legend(loc='best', fontsize='small')
            axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
            current_ax_idx +=1
            
    axs[-1].set_xlabel('Temps (secondes)') # Label X sur le dernier subplot
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    
    plot_save_path_str = config_sortie.get('results_plot_path')
    if plot_save_path_str:
        path_obj = Path(plot_save_path_str).resolve()
        path_obj.parent.mkdir(parents=True, exist_ok=True)
        try: plt.savefig(path_obj, dpi=150)
        except Exception as e: logger.error(f"Erreur sauvegarde graphique combiné: {e}", exc_info=True)
    try: plt.show()
    except Exception as e: logger.warning(f"Impossible d'afficher graphique combiné: {e}")
    plt.close(fig)

def plot_individual_run(df_resultats, nom_jeu, config_sortie):
    if df_resultats.empty or df_resultats['PV'].isnull().all(): return
    
    has_dist1 = 'Dist1' in df_resultats.columns
    has_dist2 = 'Dist2' in df_resultats.columns
    num_subplots = 2
    if has_dist1: num_subplots +=1
    if has_dist2 and has_dist1 and 'Dist2' != 'Dist1' : num_subplots +=1
    elif has_dist2 and not has_dist1: num_subplots +=1

    fig, axs = plt.subplots(num_subplots, 1, figsize=(14, 4 * num_subplots), sharex=True)
    if num_subplots == 1 : axs = [axs]

    current_ax_idx = 0
    axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['PV'], label='PV', linewidth=1.5)
    axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['SP'], 'k--', label='SP', linewidth=1.5)
    axs[current_ax_idx].set_ylabel('PV / SP')
    axs[current_ax_idx].legend(); axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
    axs[current_ax_idx].set_title(f'Simulation : {nom_jeu}')
    current_ax_idx += 1

    axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['MV'], label='MV', color='darkorange', linewidth=1.5)
    axs[current_ax_idx].set_ylabel('MV'); axs[current_ax_idx].legend(); axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
    current_ax_idx += 1
    
    if has_dist1:
        axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['Dist1'], label='Dist1', color='purple', linestyle='-.')
        axs[current_ax_idx].set_ylabel('Dist1'); axs[current_ax_idx].legend(); axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
        current_ax_idx += 1
    if has_dist2:
         if not (has_dist1 and 'Dist2' == 'Dist1'):
            axs[current_ax_idx].plot(df_resultats['Time'], df_resultats['Dist2'], label='Dist2', color='brown', linestyle=':')
            axs[current_ax_idx].set_ylabel('Dist2'); axs[current_ax_idx].legend(); axs[current_ax_idx].grid(True, linestyle=':', alpha=0.7)
            current_ax_idx += 1

    axs[-1].set_xlabel('Temps (secondes)')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plot_dir_str = config_sortie.get('individual_runs_plot_dir')
    if plot_dir_str:
        dir_path = Path(plot_dir_str).resolve(); dir_path.mkdir(parents=True, exist_ok=True)
        nom_fichier_safe = "".join(c if c.isalnum() or c in ['_','-'] else "_" for c in nom_jeu)
        plot_path = dir_path / f"run_{nom_fichier_safe}.png"
        try: plt.savefig(plot_path, dpi=120)
        except Exception as e: logger.error(f"Erreur sauvegarde graphique individuel {nom_jeu}: {e}", exc_info=True)
    else: 
        try: plt.show()
        except Exception as e: logger.warning(f"Impossible d'afficher graphique individuel {nom_jeu}: {e}")
    plt.close(fig)

# --- Exécution Principale (pas de changement majeur ici, sauf si vous voulez modifier comment les jeux sont parsés) ---
# ... (collez ici votre fonction main() complète et correcte)
def main():
    config_file = f"{NOM_SCRIPT_SANS_EXTENSION}.ini"
    config = load_config_and_setup_logging(config_file)
    try:
        modele_procede, scaler_X, scaler_y = load_process_model_and_scalers(config['ModeleProcede'])
        noms_features_modele = []
        if hasattr(scaler_X, 'feature_names_in_'):
            noms_features_modele = scaler_X.feature_names_in_.tolist()
            logger.info(f"Noms features modèle (depuis scaler_X): {noms_features_modele}")
        else:
            logger.warning("scaler_X sans 'feature_names_in_'. Fallback config (moins robuste).")
            noms_features_modele = get_model_feature_names_from_config(config['ModeleProcede'])
        if not noms_features_modele: logger.error("Noms features modèle indéterminés. Arrêt."); return

        jeux_de_reglage = []
        if config.has_section('JeuxDeReglagePID'):
            for cle_jeu, str_val_jeu in config.items('JeuxDeReglagePID'):
                parts = [p.strip() for p in str_val_jeu.split(',')]
                try:
                    kp_val = float(parts[0])
                    try: ti_val = float(parts[1])
                    except ValueError: ti_val = float('inf') if parts[1].lower() == 'inf' else (_ for _ in ()).throw(ValueError(f"Ti invalide: {parts[1]}"))
                    td_val = float(parts[2])
                    nom_jeu_val = parts[3] if len(parts) > 3 else cle_jeu
                    jeux_de_reglage.append({'nom': nom_jeu_val, 'Kp': kp_val, 'Ti': ti_val, 'Td': td_val})
                except (ValueError, IndexError) as e: logger.error(f"Erreur parse JeuxDeReglagePID '{cle_jeu}': '{str_val_jeu}'. {e}. Ignoré.")
        if not jeux_de_reglage: logger.error("Aucun jeu de réglage PID valide. Arrêt."); return
        
        tous_dfs_resultats_simulation = []
        resume_toutes_metriques = []
        noms_jeux_pour_plot_global = []
        for jeu_pid in jeux_de_reglage:
            df_resultats = run_closed_loop_simulation(config, jeu_pid['nom'], jeu_pid['Kp'], jeu_pid['Ti'], jeu_pid['Td'],
                                                 modele_procede, scaler_X, scaler_y, noms_features_modele)
            tous_dfs_resultats_simulation.append(df_resultats)
            noms_jeux_pour_plot_global.append(jeu_pid['nom'])
            if not df_resultats.empty and not df_resultats['PV'].isnull().all():
                tsamp_s_main = config.getfloat('ParametresPIDBase', 'tsamp_pid_sim_ms') / 1000.0
                metriques = calculate_performance_metrics(df_resultats, tsamp_s_main)
                metriques.update({'nom_jeu': jeu_pid['nom'], 'Kp': jeu_pid['Kp'], 'Ti': jeu_pid['Ti'], 'Td': jeu_pid['Td']})
                resume_toutes_metriques.append(metriques)
                if config.getboolean('Sortie', 'plot_individual_runs', fallback=False):
                    plot_individual_run(df_resultats, jeu_pid['nom'], config['Sortie'])
            else:
                logger.warning(f"Simu {jeu_pid['nom']} vide/PV NaN. Métriques non calculées.")
                nan_metrics = {'nom_jeu': jeu_pid['nom'], 'Kp': jeu_pid['Kp'], 'Ti': jeu_pid['Ti'], 'Td': jeu_pid['Td'],
                               'IAE': np.nan, 'ISE': np.nan, 'ITAE': np.nan, 'Overshoot': np.nan, 
                               'TempsStabilisation': np.nan, 'TempsMontee': np.nan}
                resume_toutes_metriques.append(nan_metrics)
        
        plot_simulation_results(tous_dfs_resultats_simulation, noms_jeux_pour_plot_global, config['Sortie'])
        if resume_toutes_metriques:
            df_metriques = pd.DataFrame(resume_toutes_metriques)
            cols_metriques_ordre = ['nom_jeu', 'Kp', 'Ti', 'Td', 'IAE', 'ISE', 'ITAE', 'Overshoot', 'TempsMontee', 'TempsStabilisation']
            cols_metriques_presentes = [col for col in cols_metriques_ordre if col in df_metriques.columns]
            df_metriques = df_metriques.reindex(columns=cols_metriques_presentes)
            path_csv_metriques_str = config.get('Sortie', 'results_metrics_csv')
            if path_csv_metriques_str:
                path_obj_csv = Path(path_csv_metriques_str).resolve(); path_obj_csv.parent.mkdir(parents=True, exist_ok=True)
                try: df_metriques.to_csv(path_obj_csv, index=False, float_format='%.3f', sep =';'); logger.info(f"Résumé métriques: {path_obj_csv}")
                except Exception as e: logger.error(f"Erreur sauvegarde CSV métriques: {e}", exc_info=True)
            print("\nRésumé des Métriques :"); print(df_metriques.to_string(float_format="%.3f"))
    except FileNotFoundError as e_fnf: logger.critical(f"Fichier requis non trouvé: {e_fnf}", exc_info=True)
    except ValueError as e_val: logger.critical(f"Erreur valeur (config?): {e_val}", exc_info=True)
    except configparser.Error as e_cfg_parser: logger.critical(f"Erreur parse config: {e_cfg_parser}", exc_info=True)
    except Exception as e_globale: logger.critical(f"Erreur inattendue principale: {e_globale}", exc_info=True)
    finally: logger.info(f"--- {NOM_SCRIPT_SANS_EXTENSION}.py terminé ---"); winsound.Beep(1000, 500)

if __name__ == "__main__":
    main()

INFO (pré-config logger): La variable __file__ n'est pas définie. Utilisation de 'PID_Tuner_Gemini' comme nom de base par défaut.
2025-05-18 21:02:44,680 - INFO - Logging configuré. Niveau: DEBUG. Fichier log: C:\Users\EDVA10053293\OneDrive - Groupe Avril\08 - Marchine Learning\Jupyter\Regulation PID\Pid_tuner\trace\pid_tuner_repro_reel.txt
2025-05-18 21:02:44,681 - INFO - Configuration chargée depuis 'PID_Tuner_Gemini.ini'.
2025-05-18 21:02:45,485 - INFO - Modèle chargé: C:\Users\EDVA10053293\OneDrive - Groupe Avril\08 - Marchine Learning\Jupyter\Regulation PID\Pid_Model\trained_TIC4401.joblib, Scalers chargés: C:\Users\EDVA10053293\OneDrive - Groupe Avril\08 - Marchine Learning\Jupyter\Regulation PID\Pid_Model\scalers_TIC4401.joblib
2025-05-18 21:02:45,486 - INFO - Noms features modèle (depuis scaler_X): ['PV_lag_1', 'PV_lag_2', 'PV_lag_3', 'PV_lag_4', 'PV_lag_5', 'PV_lag_6', 'PV_lag_7', 'PV_lag_8', 'PV_lag_9', 'PV_lag_10', 'MV_lag_1', 'MV_lag_2', 'MV_lag_3', 'MV_lag_4', 'MV_lag_5', 

C:\Users\EDVA10053293\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\EDVA10053293\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\EDVA10053293\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:380: InconsistentVersion

2025-05-18 21:02:45,686 - DEBUG - Étape 3: input_X_df_modele (avant scale):
    PV_lag_1   PV_lag_2   PV_lag_3  PV_lag_4  PV_lag_5  PV_lag_6  PV_lag_7  PV_lag_8  PV_lag_9  PV_lag_10  MV_lag_1  MV_lag_2  MV_lag_3  MV_lag_4  MV_lag_5  MV_lag_6  MV_lag_7  MV_lag_8  MV_lag_9  MV_lag_10  SP_lag_1  SP_lag_2  SP_lag_3  SP_lag_4  SP_lag_5  SP_lag_6  SP_lag_7  SP_lag_8  SP_lag_9  SP_lag_10  Dist1_lag_1  Dist1_lag_2  Dist1_lag_3  Dist1_lag_4  Dist1_lag_5  Dist1_lag_6  Dist1_lag_7  Dist1_lag_8  Dist1_lag_9  Dist1_lag_10
0  51.429859  51.676447  52.289758      60.0      60.0      60.0      60.0      60.0      60.0       60.0       0.0     100.0  29.59129     29.43     29.43     29.43     29.43     29.43     29.43      29.43      90.0      90.0      90.0      90.0      90.0      90.0      90.0      90.0      90.0       90.0          1.0          1.0          1.0          1.0          1.0          1.0          1.0          1.0          1.0           1.0
2025-05-18 21:02:45,688 - DEBUG - Étape 3: inp